In [5]:
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import date
from datetime import timedelta
import json

In [6]:
def findMonday(basedate):
    # base = datetime.strptime(basedate, "%Y%m%d").date()
    interval = (7 - date.weekday(basedate)) % 7
    startdate = basedate + timedelta(days=interval)
    return startdate

def day_reset():  # 메인함수

    fromdate_temp = datetime.today() - timedelta(days=7)
    fromdate = fromdate_temp.strftime("%Y%m%d")  # 지난주
    #todate = datetime.today().strftime("%Y%m%d")  # '20161128' 이번주
    startdate = findMonday(datetime.strptime(fromdate, "%Y%m%d").date())- timedelta(days=7)
    enddate = startdate + timedelta(days=6)
    startdate=startdate.strftime("%Y%m%d")
    enddate=enddate.strftime("%Y%m%d")
    return startdate,enddate

def len_test (col1, col2) :
    if len(col1)== len(col2) : return "T"
    else : return "F"

def calculator_scale_1st (scale,graph_data,graph_period,start_date):
    basedate = '20191104'
    #이날 있으면 1그룹 없으면 2그룹
    if basedate in graph_period and start_date in graph_period :
        base_idx = graph_period.index(basedate)
        idx = graph_period.index(start_date)
        return scale * graph_data[idx] / graph_data[base_idx]
    else : return None
    
def calweek_calculator(date):  # 날짜를 넣으면 주차를 반환 (yyyymmdd)
    today_datetime = datetime.datetime.strptime(date, '%Y%m%d')  # w_date의 날짜
    today_iso = today_datetime.isoweekday()  # w_date의 요일 (월1 화2 수3 목4 금5 토6 일7)
    reference_date = today_datetime + datetime.timedelta(days=(4-today_iso))  # 주차를 정하는 기준날짜 (목요일의 날짜)
    reference_date_str = datetime.datetime.strftime(reference_date, '%Y%m%d')  # 기준날짜의 string

    first_date = datetime.date(reference_date.year, reference_date.month, 1)  # 기준날짜 포함 월의 첫날
    mon_date_list = [first_date + datetime.timedelta(days=i) for i in range(calendar.monthrange(reference_date.year, reference_date.month)[1])]  # 기준날짜 포함 월의 모든 일자 리스트
    week = [datetime.datetime.strftime(date, '%Y%m%d') for date in mon_date_list if date.isocalendar()[2] == 4].index(reference_date_str)+1  # 기준날짜 포함월의 모든 목요일 중 기준날짜가 몇번째 목요일인가?
    calweek = str(reference_date.month) + '월 ' + str(week) + '주'

    return calweek

def yearmonth_calculator(date):  # 날짜를 넣으면 주차를 반환 (yyyymmdd)
    today_datetime = datetime.strptime(date, '%Y%m%d')  # w_date의 날짜
    today_iso = today_datetime.isoweekday()  # w_date의 요일 (월1 화2 수3 목4 금5 토6 일7)
    reference_date = today_datetime + timedelta(days=(4-today_iso))  # 주차를 정하는 기준날짜 (목요일의 날짜)
    reference_date_str = datetime.strftime(reference_date, '%Y%m%d')  # 기준날짜의 string
    return reference_date.strftime("%Y%m")

def keyword_cleansing (i,json_txt):
    try  :
        json_txt=json.loads(json_txt)
        return json_txt[i]
    except : return {'title':None,'data':None}

In [7]:
try:
    conn = psycopg2.connect(host='10.123.184.193', dbname='edw', user='sys', password='eos40d')
    cur = conn.cursor(cursor_factory=RealDictCursor)
except Exception:
    print('Fail to connect DB')

In [10]:
###이번주 새로 수집된 데이터 불러오기 df1
#sql = 'select * from sbd.stg_sbds_new_search_cnt_week;'
#cur.execute(sql)
#result = cur.fetchall()
#df = pd.DataFrame(result)

df=pd.read_csv(f'D:/work/_jeong_jaekeun/naver/new_keyword_crawl/result_data/grp_1st/keyword_crawling2_20170102.txt',header=None, sep = "\t", error_bad_lines=False)
df33=pd.read_csv(f'D:/work/_jeong_jaekeun/naver/new_keyword_crawl/result_data/grp_1st/keyword_crawling3_20170102.txt',header=None, sep = "\t", error_bad_lines=False)
df=pd.concat([df, df33],ignore_index=True)


In [ ]:
##5개 키워드 분리
result_df=pd.DataFrame()
for i in range(0,5) :
    #df_name = "df_"+str(i)
    tmp_df=pd.DataFrame()  
    tmp_df['keyword']=df.apply(lambda x: keyword_cleansing(i,x[0])['title'],axis = 1)
    tmp_df['data']=df.apply(lambda x: keyword_cleansing(i,x[0])['data'],axis = 1)
    #tmp_df['cluster_id'] = df
    result_df=pd.concat([result_df,tmp_df])
    print(len(result_df))
result_df['cluster_id'] = result_df.index

90669


In [9]:
tt_df

,0
0,"[{""title"":""부산폭행"",""data"":[{""period"":""20170109"",..."
1,"[{""title"":""트라이애슬론"",""data"":[{""period"":""20170102..."
2,"[{""title"":""짐바브웨"",""data"":[{""period"":""20170102"",..."
3,"[{""title"":""6.25"",""data"":[{""period"":""20170102"",..."
4,"[{""title"":""이마트킹크랩"",""data"":[{""period"":""20170102..."
...,...
90664,"[{""title"":""일본제모패드"",""data"":[{""period"":""20170116..."
90665,"[{""title"":""에뜨와오리털패딩"",""data"":[{""period"":""201812..."
90666,"[{""title"":""타임모직자켓"",""data"":[{""period"":""20191104..."
90667,"[{""title"":""아치보호대궁"",""data"":[{""period"":""20180827..."


In [339]:
###1st 계산을 위한 존데이터 기준날짜 20191104의 scale조회하기 df2

sql = "select * from sbd.sbds_search_cnt_week where start_date = '20191104'"
cur.execute(sql)
result = cur.fetchall()
df2 = pd.DataFrame(result)
df2=df2.rename(columns={'start_date':'stand_start_date','scale':'stand_scale' })

In [340]:
##수집된 데이터의 graph_period와graph_data 개수 맞는 지 확인 후 df1, df2결합 

#result_df['test'] = result_df.apply(lambda x: len_test(x['graph_data'],x['graph_period']),axis=1)
#result_df=result_df[result_df['test']=="T"]
merged_df = pd.merge(result_df,df2, on="keyword",how="left")
merged_df.head(10)
#merged_df[['cluster_id','keyword','graph_data','graph_period','w_date','stand_start_date','stand_scale']].head(2)

,keyword,data,cluster_id,stand_start_date,end_date,year_month,stand_scale,indate
0,부산폭행,"[{'period': '20170109', 'value': 0.00788}, {'p...",0,2019-11-04,2019-11-10,201911,60,2019-11-11
1,트라이애슬론,"[{'period': '20170102', 'value': 0.27361}, {'p...",1,2019-11-04,2019-11-10,201911,808,2019-11-11
2,짐바브웨,"[{'period': '20170102', 'value': 0.23178}, {'p...",2,2019-11-04,2019-11-10,201911,1483,2019-11-11
3,6.25,"[{'period': '20170102', 'value': 0.19519}, {'p...",3,2019-11-04,2019-11-10,201911,1714,2019-11-11
4,이마트킹크랩,"[{'period': '20170102', 'value': 0.17287}, {'p...",4,2019-11-04,2019-11-10,201911,578,2019-11-11
5,이마트성수점,"[{'period': '20170102', 'value': 0.55708}, {'p...",5,2019-11-04,2019-11-10,201911,1454,2019-11-11
6,11월11일,"[{'period': '20170102', 'value': 0.00803}, {'p...",6,2019-11-04,2019-11-10,201911,23673,2019-11-11
7,설명절인사말,"[{'period': '20170102', 'value': 0.08853}, {'p...",7,2019-11-04,2019-11-10,201911,7,2019-11-11
8,이천제일고등학교,"[{'period': '20170102', 'value': 0.18144}, {'p...",8,2019-11-04,2019-11-10,201911,1073,2019-11-11
9,어린이날,"[{'period': '20170102', 'value': 0.16253}, {'p...",9,2019-11-04,2019-11-10,201911,1354,2019-11-11


In [341]:
###계산하기
###기준일자 
stand_start_date = '20191104'
###뽑을 날짜
return_start_date, return_end_date = ('20201207','20201213')

date_scale = return_start_date+"_scale"
merged_df[date_scale] = merged_df.apply(lambda x: caculator_1st(x['data'],x['stand_scale'],stand_start_date,return_start_date),axis =1)

w_date = datetime.now().strftime("%Y%m%d")
merged_df['start_date']= return_start_date
merged_df['end_date']=return_end_date
merged_df['indate'] =w_date
merged_df['yearmonth'] = yearmonth_calculator(return_start_date)
merged_df.head(10)

print(return_start_date)
merged_df[return_start_date+'ratio'] = merged_df.apply(lambda x: [i['value'] for i in x['data'] if i['period']==return_start_date], axis =1)

cluster_df=merged_df.groupby('group_id')['data','stand_scale'].first()
cluster_df['group_stand_ratio']=cluster_df.apply(lambda x: [i['value'] for i in x['data'] if i['period']==stand_start_date], axis =1)
cluster_df=cluster_df.rename(columns ={'stand_scale':'group_stand_scale'})

final_df=pd.DataFrame()
final_df= pd.merge(merged_df,cluster_df, how='outer',on='group_id')
final_df.head(10)

final_df[return_start_date+'ratio']= final_df.apply(lambda x: x[return_start_date+'ratio'][0] if (len(x[return_start_date+'ratio'])==1) else 0, axis =1)
final_df['group_stand_ratio']= final_df.apply(lambda x: x['group_stand_ratio'][0] if (len(x['group_stand_ratio'])==1) else 0, axis =1)
final_df[date_scale+"new"]=final_df.apply(lambda x: x['group_stand_scale']*x[return_start_date+'ratio']/x['group_stand_ratio'] if (x['group_stand_ratio']>0) else 0, axis = 1)

final_df=final_df.rename(columns = {date_scale+"new" : 'scale'})
#final_df.head(2)
final_df=final_df[['keyword','start_date','end_date','yearmonth','scale','indate']].head(20)
final_df=final_df.astype({'scale' : 'int'})
try:
    conn = psycopg2.connect(host='10.123.184.193', dbname='edw', user='sys', password='eos40d')
    cur = conn.cursor(cursor_factory=RealDictCursor)
except Exception:
    print('Fail to connect DB')
output = io.StringIO()
# ignore the index

tbl='sbd.sbds_search_cnt_week'
final_df.to_csv(output, header=False, index=False)
output.seek(0)
cur.copy_expert("COPY %s FROM STDIN WITH CSV" % tbl, output)
conn.commit()
cur.close()
